# **Entrenamiento y evaluación de un sistema de identificación de detección de malezas en el cultivo de tomate usando RetinaNet**

_________________________________________________________________________________________________________________________________________________________________________________________________________
Angel Toledo Flores<br>
Ingeniero Mecánico Agrícola/Ingeniero en Sistemas Computacionales.<br>
**E-mail**: contactoangeltoledo@gmail.com<br>
 [Linkedin profile](https://www.linkedin.com/in/angel-toledo/) | [Portafolio de proyectos](https://angeltoledo.ga) <br> 
 [ DataSet: CSIC-CAR ](#)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Inicialización de Retina NET

In [ ]:
#Importar librerías

import os
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas as pd 
import matplotlib.pyplot as plt
import time

from google.colab import drive
from google.colab import files


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Fri May 20 15:54:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Aceso drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 6209, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6209 (delta 0), reused 2 (delta 0), pack-reused 6205
Receiving objects: 100% (6209/6209), 13.48 MiB | 22.40 MiB/s, done.
Resolving deltas: 100% (4201/4201), done.


In [ ]:
#%cd /content/drive/MyDrive/Tomato_Weed_Detection/keras-retinanet
%cd /content/keras-retinanet
!pip install .
!python setup.py build_ext --inplace

/content/keras-retinanet
Processing /content/keras-retinanet
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=169495 sha256=2327b9295898566c682e49c82fd0d4c917596b3052c9900ef302b48175340cc9
  Stored in directory: /root/.cache/pip/wheels/32/29/34/9b33c07f08b1be9e77607c1fc6b08c679489aa7ddaed329652
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=a731f7112f0a3a0dfaae0b3b5f7ca8f4fff90c939dc65dbfa73357a74fa25c4b
  Stored in directory: /root/.cache/pip/wheels/bd/ef/06/5d65f696360436c3a423020c4b7fd8c558

In [ ]:
# Configurar retina net

%matplotlib inline
%reload_ext autoreload
%autoreload 2

import keras
import tensorflow as tf
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## División de dataset

In [ ]:
#Copiar dataset al entorno de trabajo
!mkdir /content/dataset/
!mkdir /content/dataset/originals
#!mkdir dataset/train dataset/val
!cp -R /content/drive/MyDrive/Tomato_Weed_Detection/almacen/* /content/dataset

^C


In [ ]:
"""
#Dividir 70% entrenamiento 30% validación
path_dataset = /content/dataset/originals
for imagen in glob.glob('/content/house/test/*JPG):
  print(imagen)""

# Entrenamiento de la red neuronal

In [ ]:
!chmod +x /content/drive/MyDrive/Tomato_Weed_Detection/keras-retinanet/keras_retinanet/bin/train.py

In [ ]:
!chmod +x /content/keras-retinanet/keras_retinanet/bin/train.py

In [ ]:
#import urllib.request # path where pre-trained weights should be saved
PRETRAINED_MODEL = '/content/drive/MyDrive/Tomato_Weed_Detection/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5'# model url
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'# retrieve model from model url and save 
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)


PRETRAINED_MODEL = '/content/keras-retinanet/snapshots/resnet50_csv_50.h5' # model url
ANOTACIONES_T = '/content/drive/MyDrive/Tomato_Weed_Detection/anotations.csv'
CLASES_T = '/content/drive/MyDrive/Tomato_Weed_Detection/clases.csv'
print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to /content/keras-retinanet/snapshots/resnet50_csv_50.h5


In [ ]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --gpu 0 --backbone 'resnet50' --weights {PRETRAINED_MODEL} --batch-size 8 --steps 10  --epochs 150 csv {ANOTACIONES_T} {CLASES_T}

1 Physical GPUs, 1 Logical GPUs
Creating model, this may take a second...
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
Model: "retinanet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1 (Conv2D)                 (None, None, None,   9408        ['input_1[0][0]']                
                                64)                                                               
                       

##  Prueba de detección

In [ ]:
!cp /content/keras-retinanet/snapshots/resnet50_csv_150.h5 /content/drive/MyDrive/Tomato_Weed_Detection
!cp /content/keras-retinanet/snapshots/resnet50_csv_140.h5 /content/drive/MyDrive/Tomato_Weed_Detection
!cp /content/keras-retinanet/snapshots/resnet50_csv_145.h5 /content/drive/MyDrive/Tomato_Weed_Detection

In [ ]:
from keras.models import load_model
from keras_retinanet import models

model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv(CLASES_T, header=None).T.loc[0].to_dict()

snapshots/resnet50_csv_99.h5


In [ ]:
test_df = pd.read_csv(ANOTACIONES_T)
test_df.head()

,/content/drive/MyDrive/Tomato_Weed_Detection/almacen/57_IMG_4663.JPG,3277,2671,3662,2981,LYPES
0,/content/drive/MyDrive/Tomato_Weed_Detection/a...,2409,2401,2807,2654,CYP_CYPRO
1,/content/drive/MyDrive/Tomato_Weed_Detection/a...,1473,2082,2014,2674,POROL
2,/content/drive/MyDrive/Tomato_Weed_Detection/a...,1627,2824,2029,3260,POROL
3,/content/drive/MyDrive/Tomato_Weed_Detection/a...,2725,1962,2782,2026,NCI
4,/content/drive/MyDrive/Tomato_Weed_Detection/a...,992,1109,1057,1158,NCI


In [ ]:
import skimage.io as io

def predict(image):
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(
     np.expand_dims(image, axis=0)
     )

    boxes /= scale

    return boxes, scores, labels

umbralScore = 0.7

def draw_detections(image, boxes, scores, labels):
      for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < umbralScore:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(image, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(image, b, caption)

In [ ]:
contador = 0
for index, row in test_df.iterrows():
    contador = contador +1
    print(row[0], index)
    image = io.imread(row[0])
    #image = io.imread('images/hand2.jpg')

    boxes, scores, labels = predict(image)

    if contador == 25:
      break
    
    draw = image.copy()
    draw_detections(draw, boxes, scores, labels)

    plt.axis('off')
    plt.imshow(draw)
    plt.show()